In [1]:
# !python style_mixing.py --outdir=_my_images/style_mixing/temp --rows=85,100,75,458,1500 --cols=55,821,1789,293 \
#     --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl

In [2]:
# Copyright (c) 2021, NVIDIA CORPORATION.  All rights reserved.
#
# NVIDIA CORPORATION and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA CORPORATION is strictly prohibited.

"""Generate style mixing image matrix using pretrained network pickle."""

import os
import re
from typing import List

import click
import dnnlib
import numpy as np
import PIL.Image
import torch

import legacy

#----------------------------------------------------------------------------

def num_range(s: str) -> List[int]:
    '''Accept either a comma separated list of numbers 'a,b,c' or a range 'a-c' and return as a list of ints.'''

    range_re = re.compile(r'^(\d+)-(\d+)$')
    m = range_re.match(s)
    if m:
        return list(range(int(m.group(1)), int(m.group(2))+1))
    vals = s.split(',')
    return [int(x) for x in vals]

#----------------------------------------------------------------------------

In [3]:
# @click.command()
# @click.option('--network', 'network_pkl', help='Network pickle filename', required=True)
# @click.option('--rows', 'row_seeds', type=num_range, help='Random seeds to use for image rows', required=True)
# @click.option('--cols', 'col_seeds', type=num_range, help='Random seeds to use for image columns', required=True)
# @click.option('--styles', 'col_styles', type=num_range, help='Style layer range', default='0-6', show_default=True)
# @click.option('--trunc', 'truncation_psi', type=float, help='Truncation psi', default=1, show_default=True)
# @click.option('--noise-mode', help='Noise mode', type=click.Choice(['const', 'random', 'none']), default='const', show_default=True)
# @click.option('--outdir', type=str, required=True)
def generate_style_mix(
    network_pkl: str,
    row_seeds: List[int],
    col_seeds: List[int],
    col_styles: List[int],
    truncation_psi: float,
    noise_mode: str,
    outdir: str,
    save_images = True,
    truncation_psi_list = []
):
    """Generate images using pretrained network pickle.

    Examples:

    \b
    python style_mixing.py --outdir=out --rows=85,100,75,458,1500 --cols=55,821,1789,293 \\
        --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl
    """
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

    outdir += '/' + network_pkl.split('/')[-1].split('.')[0]    ## victor
    os.makedirs(outdir, exist_ok=True)

    print('Generating W vectors...')
    all_seeds = list(set(row_seeds + col_seeds))
    all_z = np.stack([np.random.RandomState(seed).randn(G.z_dim) for seed in all_seeds])
    all_w = G.mapping(torch.from_numpy(all_z).to(device), None)
    w_avg = G.mapping.w_avg
    all_w = w_avg + (all_w - w_avg) * truncation_psi
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))}

    print('Generating images...')
    all_images = G.synthesis(all_w, noise_mode=noise_mode)
    all_images = (all_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].clone()
            w[col_styles] = w_dict[col_seed][col_styles]
            image = G.synthesis(w[np.newaxis], noise_mode=noise_mode)
            image = (image.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
            image_dict[(row_seed, col_seed)] = image[0].cpu().numpy()

    if save_images:
        print('Saving images...')
        os.makedirs(outdir, exist_ok=True)
        for (row_seed, col_seed), image in image_dict.items():
            PIL.Image.fromarray(image, 'RGB').save(f'{outdir}/{row_seed}-{col_seed}.png')
            
            
    if not truncation_psi_list:
        truncation_psi_list = [truncation_psi]            
    
    print('truncation_psi_list:',truncation_psi_list)
    
    print('(len(col_seeds) + 1):',(len(col_seeds) + 1))
    print('(len(row_seeds) + 1):',(len(row_seeds) + 1))
    print('len(truncation_psi_list)*(len(row_seeds) + 1):',len(truncation_psi_list)*(len(row_seeds) + 1))
    

    print('Saving image grid...')
    W = G.img_resolution
    H = G.img_resolution
#     canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * len(truncation_psi_list) * (len(row_seeds) + 1)), 'black')
    
#     for truncation_psi in truncation_psi_list:
    for psi_index, truncation_psi in enumerate(truncation_psi_list):
        for row_idx, row_seed in enumerate([0] + row_seeds):
            for col_idx, col_seed in enumerate([0] + col_seeds):
                row_idx_new = psi_index*(len(row_seeds) + 1) + row_idx
                print(psi_index,row_idx,row_idx_new,col_idx)
                if row_idx == 0 and col_idx == 0:
                    continue
                key = (row_seed, col_seed)
                if row_idx == 0:
                    key = (col_seed, col_seed)
                if col_idx == 0:
                    key = (row_seed, row_seed)
#                 canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx))
                canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx_new))
            
    grid_name = 'grid'
    grid_name += '__rs:' + '-'.join([str(x) for x in row_seeds])
    grid_name += '__cs:' + '-'.join([str(x) for x in col_seeds])
    grid_name += '__csty:' + '-'.join([str(x) for x in col_styles])
#     grid_name += '__psi:' + str(truncation_psi)
    grid_name += '__psy:' + '-'.join([str(x) for x in truncation_psi_list])
    grid_name += '__noise:' + noise_mode
    print(f'{outdir}/{grid_name}.png')
    canvas.save(f'{outdir}/{grid_name}.png')

In [4]:
network_pkl = 'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl'    
# row_seeds = [1,750,1500]
row_seeds = [1]
# col_seeds = [400,750,1000]
col_seeds = [1,2,3,4,5]
col_styles = [0,1,2,3,4,5,6]
truncation_psi = 1.0
noise_mode = 'const'
outdir = '_my_images/style_mixing'
save_images = False

# col_styles = list(range(18))
# truncation_psi = 0.0
truncation_psi_list = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images,truncation_psi_list)

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/metfaces.pkl"...
Generating W vectors...
Setting up PyTorch plugin "bias_act_plugin"... 

/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py:283: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) is not compatible with the compiler Pytorch was
built with for this platform, which is g++ on linux. Please
use g++ to to compile your extension. Alternatively, you may
compile PyTorch from source using c++, and then you can also use
c++ to compile your extension.

See https://github.com/pytorch/pytorch/blob/master/CONTRIBUTING.md for help
with compiling PyTorch from source.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(WRONG_COMPILER_WARNING.format(


Failed!
Generating images...
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1667, in _run_ninja_build
    subprocess.run(
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/subprocess.py", line 512, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['ninja', '-v']' returned non-zero exit status 1.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/petrobr/home/victor

/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py:283: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) is not compatible with the compiler Pytorch was
built with for this platform, which is g++ on linux. Please
use g++ to to compile your extension. Alternatively, you may
compile PyTorch from source using c++, and then you can also use
c++ to compile your extension.

See https://github.com/pytorch/pytorch/blob/master/CONTRIBUTING.md for help
with compiling PyTorch from source.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(WRONG_COMPILER_WARNING.format(


...

Generating style-mixed images...
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTo

/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1079, in load
    return _jit_compile(
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1317, in _jit_compile
    retur

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
truncation_psi_list: [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
(len(col_seeds) + 1): 6
(len(row_seeds) + 1): 2
len(truncation_psi_list)*(len(row_seeds) + 1): 12
Sa

/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1079, in load
    return _jit_compile(
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1317, in _jit_compile
    retur

0 1 1 0
0 1 1 1
0 1 1 2
0 1 1 3
0 1 1 4
0 1 1 5
1 0 2 0
1 0 2 1
1 0 2 2
1 0 2 3
1 0 2 4
1 0 2 5
1 1 3 0
1 1 3 1
1 1 3 2
1 1 3 3
1 1 3 4
1 1 3 5
2 0 4 0
2 0 4 1
2 0 4 2
2 0 4 3
2 0 4 4
2 0 4 5
2 1 5 0
2 1 5 1
2 1 5 2
2 1 5 3
2 1 5 4
2 1 5 5
3 0 6 0
3 0 6 1
3 0 6 2
3 0 6 3
3 0 6 4
3 0 6 5
3 1 7 0
3 1 7 1
3 1 7 2
3 1 7 3
3 1 7 4
3 1 7 5
4 0 8 0
4 0 8 1
4 0 8 2
4 0 8 3
4 0 8 4
4 0 8 5
4 1 9 0
4 1 9 1
4 1 9 2
4 1 9 3
4 1 9 4
4 1 9 5
5 0 10 0
5 0 10 1
5 0 10 2
5 0 10 3
5 0 10 4
5 0 10 5
5 1 11 0
5 1 11 1
5 1 11 2
5 1 11 3
5 1 11 4
5 1 11 5
_my_images/style_mixing/metfaces/grid__rs:1__cs:1-2-3-4-5__csty:0-1-2-3-4-5-6__psy:0.0-0.2-0.4-0.6-0.8-1.0__noise:const.png


In [8]:
list(range(8,18))

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [17]:
dataset_name = 'ffhq'
network_pkl = f'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/{dataset_name}.pkl'    
row_seeds = [0]
col_seeds = [1,2,3,4,5]
col_styles = [0,2,4,6,8,10,12,14,16]
truncation_psi = 1.0
noise_mode = 'const'
outdir = '_my_images/style_mixing'
save_images = False

generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images)

Loading networks from "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"...
Generating W vectors...
Generating images...
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up Py

/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1079, in load
    return _jit_compile(
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1317, in _jit_compile
    retur

Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "/petrobr/home/victor.ferreira/my_github/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 110, in get_plugin
    torch.utils.cpp_extension.load(name=module_name, verbose=verbose_build, sources=sources, **build_kwargs)
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1079, in load
    return _jit_compile(
  File "/petrobr/home/victor.ferreira/anaconda3/lib/python3.8/site-packages/torch/utils/cpp_extension.py", line 1317, in _jit_compile
    retur

0 0 0 5
0 1 1 0
0 1 1 1
0 1 1 2
0 1 1 3
0 1 1 4
0 1 1 5
_my_images/style_mixing/ffhq/grid__rs:0__cs:1-2-3-4-5__csty:0-2-4-6-8-10-12-14-16__psy:1.0__noise:const.png


In [ ]:
w_dict.keys()

In [ ]:
col_styles = [17]
w = w_dict[1]
w[col_styles]
# w[0]

In [ ]:
w_dict[1][np.newaxis].shape

In [ ]:
dataset_name = 'brecahad'
network_pkl = f'https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/{dataset_name}.pkl'    
row_seeds = [7]
col_seeds = [7,8,9,10,11,12,13,14,15,16]
col_styles = [0,1,2,3,4,5,6]
truncation_psi = 1.0
noise_mode = 'const'
outdir = '_my_images/style_mixing'
save_images = False

# generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images)

In [ ]:
for i in range(7):
    col_styles = list(range(i+1))
    print('col_styles:',col_styles)
    generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images)
    
    col_styles = [i]
    print('col_styles:',col_styles)
    generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images)

In [ ]:
for truncation_psi in [round(x,1) for x in np.linspace(0,1,6)]:
    print('truncation_psi:',truncation_psi)
    generate_style_mix(network_pkl,row_seeds,col_seeds,col_styles,truncation_psi,noise_mode,outdir,save_images)